In [ ]:
import os
os.environ["TRANSFORMERS_NO_FAST_TOKENIZER"] = "1"

In [ ]:
!pip -q install -U sentencepiece protobuf

In [ ]:
!pip -q install -U datasets transformers accelerate peft bitsandbytes safetensors sentencepiece pandas huggingface_hub
!pip -q install soundfile sherpa-onnx onnxruntime-gpu

from transformers import (
    AutoModelForSpeechSeq2Seq,
    AutoProcessor,
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    AutoModelForSeq2SeqLM
)
from datasets import load_dataset, Audio
from sklearn.metrics import accuracy_score, f1_score
import os, json, re, gc, torch
import pandas as pd
from huggingface_hub import login
from peft import PeftModel
from google.colab import drive
import glob, subprocess, numpy as np, soundfile as sf
import sherpa_onnx
import librosa
import random

os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.7/520.7 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 13.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 3.0.1 which is incompatible.
cudf-cu12 25.10.0 requires pandas<2.4.0dev0,>=2.0, but you have pandas 3.0.1 which is incompatible.
db-dtypes 1.5.0 requires pandas<3.0.0,>=1.5.3, but you have pandas 3.0.1 which is incompatible.
bqplot 0.12.45 requires pandas<3.0.0,>=1.0.0, but you have p

# **Load data** (human)

In [ ]:
IDX = [631, 1999, 626, 957, 538, 614, 42, 46, 2097, 2006]

data_test = load_dataset("leduckhai/Sentiment-Reasoning", split="test")
data_test = data_test.cast_column("audio", Audio(sampling_rate=16000))
subset = data_test.select([int(i) for i in IDX])

keep_cols = ["human_justification", "text", "label", "audio"]
missing = [c for c in keep_cols if c not in subset.column_names]
if missing:
    raise ValueError(f"Thiếu cột {missing}. Hiện có: {subset.column_names}")

subset = subset.remove_columns([c for c in subset.column_names if c not in keep_cols])
df = subset.to_pandas()

print("DF shape:", df.shape)
print("DF columns:", df.columns.tolist())
display(df.head(10))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/114M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/44.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5612 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2182 [00:00<?, ? examples/s]

DF shape: (10, 4)
DF columns: ['human_justification', 'label', 'text', 'audio']


,human_justification,label,text,audio
0,An toan va khỏe mạnh,positive,ý thức và thói quen có lợi cho việc nâng cao s...,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...
1,mô tả khách quan về số lượng ngày càng tăng v...,positive,bệnh lý ở mái tóc thì khá là ít còn ngày nay k...,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...
2,tìm kiếm thông tin,positive,bệnh tình như thế nào xin hỏi với cái dạng bện...,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...
3,"tra hỏi, đòi hỏi",negative,thay vì thuốc tiêm được hay không tức là dùng ...,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...
4,dấu hiệu cảm xúc tiêu cực,negative,méo một bên hay là bị mớm một bên hay là bị xấ...,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...
5,quan tâm và lo lắng,negative,xuất hiện như vậy thì cái cách xử lý cái những...,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...
6,thông tin thuần túy,neutral,chúng ta sẽ lựa chọn ra một số câu hỏi mà nãy ...,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...
7,mô tả khách quan,neutral,bệnh phẫu thuật tiêu hóa làm sao để có thể nhậ...,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...
8,mô tả khách quan,neutral,trả lại cho họ chất lượng cuộc sống bình thườn...,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...
9,mô tả khách quan,neutral,đó là những cái nhóm nguyên nhân chính và nhữn...,{'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x...


In [ ]:
human_text = []
for i in range(len(df)):
  human_text.append(df["text"][i])

true_labels_human = [data_test[i]["label"] for i in IDX]

# **ASR**

## Load data

In [ ]:
# ds = load_dataset("leduckhai/Sentiment-Reasoning")
# test_ds = ds["test"]

# random.seed(42)
# pos_indices = [i for i in range(len(test_ds)) if test_ds[i]["label"] == "positive"]
# neg_indices = [i for i in range(len(test_ds)) if test_ds[i]["label"] == "negative"]
# neu_indices = [i for i in range(len(test_ds)) if test_ds[i]["label"] == "neutral"]
# sample_ids = (
#     random.sample(pos_indices, 3) +
#     random.sample(neg_indices, 3) +
#     random.sample(neu_indices, 4)
# )
# random.shuffle(sample_ids)
# sample_ids

# sample_ids = [794, 1088, 745, 1679, 519, 208, 553, 1672, 79, 1558]

# === KHÔNG CHẠY LẠI Ô NÀY ĐỂ GIỮ OUTPUT ===

[794, 1088, 745, 1679, 519, 208, 553, 1672, 79, 1558]

In [ ]:
ds = load_dataset("leduckhai/Sentiment-Reasoning")
test_ds = ds["test"]

# random.seed(42)
# pos_indices = [i for i in range(len(test_ds)) if test_ds[i]["label"] == "positive"]
# neg_indices = [i for i in range(len(test_ds)) if test_ds[i]["label"] == "negative"]
# neu_indices = [i for i in range(len(test_ds)) if test_ds[i]["label"] == "neutral"]
# sample_ids = (
#     random.sample(pos_indices, 3) +
#     random.sample(neg_indices, 3) +
#     random.sample(neu_indices, 4)
# )
# random.shuffle(sample_ids)
# sample_ids

sample_ids = [794, 1088, 745, 1679, 519, 208, 553, 1672, 79, 1558]

In [ ]:
audio_samples = [test_ds[i]["audio"] for i in sample_ids]
true_labels = [test_ds[i]["label"] for i in sample_ids]

## PhoWhisper

In [ ]:
phowhisper_model = "vinai/PhoWhisper-large"

asr_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    phowhisper_model,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to(device)

processor = AutoProcessor.from_pretrained(phowhisper_model)

asr = pipeline(
    "automatic-speech-recognition",
    model=asr_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    device=0 if torch.cuda.is_available() else -1
)

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

Exception in thread Thread-auto_conversion:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_http.py", line 657, in hf_raise_for_status
    response.raise_for_status()
  File "/usr/local/lib/python3.12/dist-packages/httpx/_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '403 Forbidden' for url 'https://huggingface.co/api/models/vinai/PhoWhisper-large/discussions?p=0'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/403

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/transformers/safetensors_conversion.p

Loading weights:   0%|          | 0/1260 [00:00<?, ?it/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [ ]:
phowhisper_transcripts = []
for audio in audio_samples:
    result = asr(audio)
    phowhisper_transcripts.append(result["text"])

phowhisper_transcripts

Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> to see related `.generate()` flags.
A custom logits pr

['trong hoàn cảnh này trẻ sẽ dễ dàng mất kiểm soát cảm xúc hành vi và khó chấp nhận sự chín.',
 'dịch cộng đồng cho nên đạt được cái chuyện là phụ vắc xin là chuyện yếu hết sức quan trọng thế thì để có.',
 'hai chín tháng tuổi các bé tự nhiên có những hành vi làm ba mẹ và người nhà rất lo sợ.',
 'trường hợp mà beta terry âm tính thì mình sẽ xem kết quả siêu âm nếu siêu âm có nhận thức.',
 'răng ít thôi nhưng lại tiêu tiêu rất là nhiều và cái giải pháp cho những người mà bị tiêu.',
 'cơ sở y tế ờ để khám và tư vấn.',
 'mong muốn là muốn thực hiện cái giải pháp tiến bộ trong vấn đề phục hồi à bị tiêu xương và hoặc là hoàn.',
 'về nguyên tắc điều trị rong kinh rong huyết thì thứ nhất là phải điều trị.',
 'địa chỉ uy tín dành cho các du khách muốn trải nghiệm loại hình du lịch y tế và không chỉ phù hợp.',
 'cả một người con để kiểm soát bà mẹ cũng không nổi vì người già sa sút tứ tệ.']

## Sherpa

In [ ]:
sherpa_transcripts = []
for p in sorted(wav_paths):
    # nạp waveform & decode
    stream = rec.create_stream()
    audio, sr = sf.read(p, dtype="float32")
    if hasattr(stream, "accept_waveform"):
        stream.accept_waveform(sr, audio)
    else:
        # fallback hiếm: ghi tạm file rồi accept_wave_file
        stream.accept_wave_file(p)
    rec.decode_stream(stream)
    text = stream.result.text if hasattr(stream, "result") else ""
    sherpa_transcripts.append(text.lower())

sherpa_transcripts

[' hoàn cảnh này trẻ sẽ dễ dàng mất kiểm soát cảm xúc hành vi và khó chấp nhận sự',
 ' dịch cộng đồng cho nên đạt được cái chuyện là phủ vaccine là cái điều hết sức quan trọng thế thì để có',
 ' chín tháng tuổi các bé tự nhiên có những hành vi làm ba mẹ và người nhà rất lo sợ',
 ' trường hợp mà btv âm tính thì mình sẽ xem kết quả siêu âm nếu siêu âm có những',
 ' răng ít thôi nhưng lại tiêu xương rất là nhiều và cái giải pháp cho những người mà bị tiêu',
 ' cơ sở y tế ờ để khám và tư vấn',
 ' mong muốn là muốn thực hiện các giải pháp tiến bộ trong vấn đề phục hồi à bị tiêu xương và hoặc là hoàn',
 ' về nguyên tắc điều trị thông kinh huyết á thì à thứ nhất là phải điều trị',
 ' địa chỉ uy tín dành cho các du khách muốn trải nghiệm loại hình du lịch y tế và không chỉ phù hợp',
 ' cả một người con để kiểm soát bà mẹ cũng không nổi vì người già sa sút trí tuệ']

# **Encoder**

In [ ]:
phobert_path = "PhoBERT"
vihealth_path = "VihealthBERT"

phobert_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
vihealth_tokenizer = AutoTokenizer.from_pretrained("demdecuong/vihealthbert-base-syllable")

phobert_model = AutoModelForSequenceClassification.from_pretrained(phobert_path).to(device)
vihealth_model = AutoModelForSequenceClassification.from_pretrained(vihealth_path).to(device)

phobert_model.eval()
vihealth_model.eval()

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/836 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

Loading weights:   0%|          | 0/201 [00:02<?, ?it/s]

Loading weights:   0%|          | 0/201 [00:04<?, ?it/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
def predict(model, tokenizer, texts):
    inputs = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        preds = torch.argmax(outputs.logits, dim=1)

    id2label = model.config.id2label
    return [id2label[p.item()] for p in preds]

In [ ]:
phobert_preds = predict(phobert_model, phobert_tokenizer, phowhisper_transcripts)
vihealth_preds = predict(vihealth_model, vihealth_tokenizer, phowhisper_transcripts)

df_results = pd.DataFrame({
    "id": sample_ids,
    "phowhisper_transcript": phowhisper_transcripts,
    "label": true_labels,
    "phobert_pred": phobert_preds,
    "vihealth_pred": vihealth_preds
})
df_results

,id,phowhisper_transcript,label,phobert_pred,vihealth_pred
0,794,trong hoàn cảnh này trẻ sẽ dễ dàng mất kiểm so...,negative,negative,negative
1,1088,dịch cộng đồng cho nên đạt được cái chuyện là ...,positive,neutral,positive
2,745,hai chín tháng tuổi các bé tự nhiên có những h...,negative,negative,negative
3,1679,trường hợp mà beta terry âm tính thì mình sẽ x...,neutral,neutral,neutral
4,519,răng ít thôi nhưng lại tiêu tiêu rất là nhiều ...,negative,negative,negative
5,208,cơ sở y tế ờ để khám và tư vấn.,neutral,neutral,neutral
6,553,mong muốn là muốn thực hiện cái giải pháp tiến...,positive,positive,positive
7,1672,về nguyên tắc điều trị rong kinh rong huyết th...,neutral,neutral,neutral
8,79,địa chỉ uy tín dành cho các du khách muốn trải...,positive,positive,positive
9,1558,cả một người con để kiểm soát bà mẹ cũng không...,neutral,negative,negative


In [ ]:
phobert_preds_ = predict(phobert_model, phobert_tokenizer, sherpa_transcripts)
vihealth_preds_ = predict(vihealth_model, vihealth_tokenizer, sherpa_transcripts)

df_results = pd.DataFrame({
    "id": sample_ids,
    "sherpa_transcript": sherpa_transcripts,
    "label": true_labels,
    "phobert_pred": phobert_preds_,
    "vihealth_pred": vihealth_preds_
})
df_results

,id,sherpa_transcript,label,phobert_pred,vihealth_pred
0,794,hoàn cảnh này trẻ sẽ dễ dàng mất kiểm soát cả...,negative,negative,negative
1,1088,dịch cộng đồng cho nên đạt được cái chuyện là...,positive,positive,positive
2,745,chín tháng tuổi các bé tự nhiên có những hành...,negative,negative,negative
3,1679,trường hợp mà btv âm tính thì mình sẽ xem kết...,neutral,neutral,neutral
4,519,răng ít thôi nhưng lại tiêu xương rất là nhiề...,negative,negative,negative
5,208,cơ sở y tế ờ để khám và tư vấn,neutral,neutral,neutral
6,553,mong muốn là muốn thực hiện các giải pháp tiế...,positive,neutral,neutral
7,1672,về nguyên tắc điều trị thông kinh huyết á thì...,neutral,neutral,neutral
8,79,địa chỉ uy tín dành cho các du khách muốn trả...,positive,positive,positive
9,1558,cả một người con để kiểm soát bà mẹ cũng khôn...,neutral,negative,negative


In [ ]:
phobert_preds_text = predict(phobert_model, phobert_tokenizer, human_text)
vihealth_preds_text = predict(vihealth_model, vihealth_tokenizer, human_text)

df_results = pd.DataFrame({
    "id": IDX,
    "human_text": human_text,
    "label": true_labels,
    "phobert_pred": phobert_preds_text,
    "vihealth_pred": vihealth_preds_text
})
df_results

,id,human_text,label,phobert_pred,vihealth_pred
0,631,ý thức và thói quen có lợi cho việc nâng cao s...,negative,positive,positive
1,1999,bệnh lý ở mái tóc thì khá là ít còn ngày nay k...,positive,positive,neutral
2,626,bệnh tình như thế nào xin hỏi với cái dạng bện...,negative,negative,neutral
3,957,thay vì thuốc tiêm được hay không tức là dùng ...,neutral,neutral,neutral
4,538,méo một bên hay là bị mớm một bên hay là bị xấ...,negative,negative,negative
5,614,xuất hiện như vậy thì cái cách xử lý cái những...,neutral,neutral,neutral
6,42,chúng ta sẽ lựa chọn ra một số câu hỏi mà nãy ...,positive,neutral,neutral
7,46,bệnh phẫu thuật tiêu hóa làm sao để có thể nhậ...,neutral,positive,neutral
8,2097,trả lại cho họ chất lượng cuộc sống bình thườn...,positive,positive,positive
9,2006,đó là những cái nhóm nguyên nhân chính và nhữn...,neutral,negative,negative


# **Encoder-Decoder**

In [ ]:
bartpho_path = "BARTPho"
vit5_path = "VIT5"
vit5_pt_path = "vit5_finetuned.pt"
vit5_base = "VietAI/vit5-base"

In [ ]:
def free_vram_strong(*objs):
    for o in objs:
        try: del o
        except: pass
    gc.collect()
    if torch.cuda.is_available():
        try:
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            torch.cuda.reset_peak_memory_stats()
        except Exception:
            pass

def normalize_label(x):
    if x is None:
        return None
    x = str(x).strip().lower()
    if x in ["positive", "pos", "2"]:
        return "positive"
    if x in ["negative", "neg", "0"]:
        return "negative"
    if x in ["neutral", "neu", "1"]:
        return "neutral"
    return x

def parse_label_rationale(text: str):
    if text is None:
        return None, None
    s = str(text).strip()

    m = re.search(r"label\s*:\s*(negative|neutral|positive)", s, flags=re.IGNORECASE)
    if m:
        label = m.group(1).lower()
    else:
        parts = s.lower().split("|", 1)
        cand = parts[0].strip()
        label = None
        for lab in ["negative", "neutral", "positive"]:
            if lab in cand:
                label = lab
                break

    m2 = re.search(r"rationale\s*:\s*(.*)", s, flags=re.IGNORECASE | re.DOTALL)
    if m2:
        rationale = m2.group(1).strip()
    else:
        parts = s.split("|", 1)
        rationale = parts[1].strip() if len(parts) == 2 else s

    return label, rationale

In [ ]:
# --- Prompt template ---
def build_seq2seq_prompt(text: str) -> str:
    return (
        "Bạn là trợ lý nhận diện cảm xúc tiếng Việt.\n"
        "Nhiệm vụ: Đọc câu tiếng Việt và dự đoán cảm xúc kèm theo giải thích ngắn gọn "
        "tại sao chọn label đó dựa trên nội dung câu.\n"
        "Định dạng output: LABEL: <negative|neutral|positive> | "
        "RATIONALE: <giải thích ngắn gọn bằng tiếng Việt>.\n\n"
        f"Văn bản: {text}"
    )

# --- Load model từ folder đã save trên Google Drive ---
def load_seq2seq_model(model_dir: str, model_name: str):
    """
    Load tokenizer + model Seq2Seq (BARTPho / ViT5) từ folder trên Drive.
    Folder này được tạo bởi trainer.save_model() nên chứa cả tokenizer.
    """
    print(f"\n{'='*50}")
    print(f"Loading {model_name} from: {model_dir}")

    # BARTPho-word cần use_fast=False; ViT5 dùng SentencePiece -> cũng ok với use_fast=False
    tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False)

    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_dir,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    ).to(device)
    model.eval()

    n_params = sum(p.numel() for p in model.parameters()) / 1e6
    print(f"  Loaded ✓  |  {n_params:.1f}M params  |  device={next(model.parameters()).device}")
    return tokenizer, model

# --- Inference 1 model trên danh sách mẫu ---
@torch.no_grad()
def predict_seq2seq(
    model_name: str,
    tokenizer,
    model,
    texts: list[str],
    idx_list,
    max_new_tokens: int = 128,
    true_labels: list[str] = None,  # optional
):
    rows = []

    # bảo vệ: nếu idx_list None thì chạy hết
    if idx_list is None:
        idx_list = list(range(len(texts)))

    for pos, idx in enumerate(idx_list):
        # Lấy đúng text theo idx nếu idx là int và nằm trong range, nếu không fallback theo pos
        if isinstance(idx, (int,)) and 0 <= idx < len(texts):
            r = texts[idx]
        else:
            # fallback: dùng theo pos, tránh crash nếu idx là string/id
            if pos >= len(texts):
                break
            r = texts[pos]

        prompt = build_seq2seq_prompt(r)

        inputs = tokenizer(
            prompt,
            max_length=512,
            truncation=True,
            return_tensors="pt",
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}  # FIX: an toàn hơn .to(device)

        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3,
        )
        gen_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

        pred_label, pred_rationale = parse_label_rationale(gen_text)

        rows.append({
            "id": int(idx) if isinstance(idx, (int,)) else idx,
            "true_label": (true_labels[pos] if true_labels is not None and pos < len(true_labels) else None),
            "pred_label": normalize_label(pred_label),
            "pred_rationale": pred_rationale,
            "raw_output": gen_text,
            "model": model_name,
        })

    return pd.DataFrame(rows)


In [ ]:
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer

def load_seq2seq_from_pt(
    pt_path: str,
    base_model_name: str,
    model_name: str,
    tokenizer_name: str = None,
):
    """
    Load a Seq2Seq model (e.g. ViT5, BARTPho) from a .pt checkpoint file.
    Returns:
        tokenizer, model  (model on `device`, eval mode, float16 on GPU)
    """
    import subprocess, sys
    # Ensure sentencepiece is available (T5/ViT5 tokenizer depends on it)
    try:
        import sentencepiece
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "sentencepiece"])

    print(f"\n{'='*50}")
    print(f"Loading {model_name} from .pt: {pt_path}")
    assert os.path.exists(pt_path), f"File not found: {pt_path}"

    checkpoint = torch.load(pt_path, map_location=device)

    dtype = torch.float16 if torch.cuda.is_available() else torch.float32

    if isinstance(checkpoint, dict):
        # Resolve state dict key
        if "model_state_dict" in checkpoint:
            state_dict = checkpoint["model_state_dict"]
        elif "state_dict" in checkpoint:
            state_dict = checkpoint["state_dict"]
        else:
            state_dict = checkpoint  # raw state dict

        model = AutoModelForSeq2SeqLM.from_pretrained(
            base_model_name,
            torch_dtype=dtype,
            ignore_mismatched_sizes=True,
        )
        missing, unexpected = model.load_state_dict(state_dict, strict=False)
        if missing:
            print(f"  [WARN] Missing keys   : {missing[:5]}{'...' if len(missing) > 5 else ''}")
        if unexpected:
            print(f"  [WARN] Unexpected keys: {unexpected[:5]}{'...' if len(unexpected) > 5 else ''}")
    else:
        # Full model object saved directly
        model = checkpoint

    model = model.to(device)
    model.eval()

    tok_src = tokenizer_name or base_model_name

    try:
        from transformers import T5Tokenizer
        tokenizer = T5Tokenizer.from_pretrained(tok_src, legacy=False)
        print("  Tokenizer loaded via T5Tokenizer ✓")
    except Exception as e1:
        print(f"  T5Tokenizer failed ({e1}), falling back to AutoTokenizer...")
        try:
            tokenizer = AutoTokenizer.from_pretrained(tok_src, use_fast=True)
        except Exception:
            tokenizer = AutoTokenizer.from_pretrained(tok_src, use_fast=False)

    n_params = sum(p.numel() for p in model.parameters()) / 1e6
    print(f"  Loaded ✓  |  {n_params:.1f}M params  |  device={next(model.parameters()).device}")
    return tokenizer, model

**BARTPHO**

In [ ]:
bartpho_tokenizer, bartpho_model = load_seq2seq_model(bartpho_path, "BARTPho")
df_bartpho = predict_seq2seq("BARTPho", bartpho_tokenizer, bartpho_model, human_text, IDX)

print("\n--- BARTPho predictions (human_text) ---")
display(df_bartpho)


--- BARTPho predictions (human_text) ---


,id,true_label,pred_label,pred_rationale,raw_output,model
0,631,positive,positive,lợi ích sức khỏe,LABEL: positive | RATIONALE: lợi ích sức khỏe,BARTPho
1,1999,positive,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
2,626,positive,neutral,câu hỏi khách quan,LABEL: neutral | RATIONALE: câu hỏi khách quan,BARTPho
3,957,negative,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
4,538,negative,negative,tình trạng bệnh lý tiêu cực,LABEL: negative | RATIONALE: tình trạng bệnh l...,BARTPho
5,614,negative,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
6,42,neutral,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
7,46,neutral,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
8,2097,neutral,positive,chất lượng cuộc sống bình thường,LABEL: positive | RATIONALE: chất lượng cuộc s...,BARTPho
9,2006,neutral,negative,tác động tiêu cực đến tóc,LABEL: negative | RATIONALE: tác động tiêu cực...,BARTPho


In [ ]:
df_bartpho = predict_seq2seq("BARTPho", bartpho_tokenizer, bartpho_model, phowhisper_transcripts, sample_ids)

print("\n--- BARTPho predictionsnn (PhoWhisper transcripts) ---")
display(df_bartpho)


--- BARTPho predictionsnn (PhoWhisper transcripts) ---


,id,true_label,pred_label,pred_rationale,raw_output,model
0,794,positive,negative,mất kiểm soát,LABEL: negative | RATIONALE: mất kiểm soát,BARTPho
1,1088,positive,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
2,745,positive,negative,lo lắng và lo lắng,LABEL: negative | RATIONALE: lo lắng và lo lắng,BARTPho
3,1679,negative,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
4,519,negative,negative,tiêu thụ quá mức,LABEL: negative | RATIONALE: tiêu thụ quá mức,BARTPho
5,208,negative,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
6,553,neutral,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
7,1672,neutral,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
8,79,neutral,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
9,1558,neutral,negative,tình trạng bệnh lý tiêu cực,LABEL: negative | RATIONALE: tình trạng bệnh l...,BARTPho


In [ ]:
df_bartpho = predict_seq2seq("BARTPho", bartpho_tokenizer, bartpho_model, sherpa_transcripts, sample_ids)

print("\n--- BARTPho predictions (Sherpa transcripts) ---")
display(df_bartpho)


--- BARTPho predictions (Sherpa transcripts) ---


,id,true_label,pred_label,pred_rationale,raw_output,model
0,794,positive,negative,mất kiểm soát,LABEL: negative | RATIONALE: mất kiểm soát,BARTPho
1,1088,positive,positive,nhấn mạnh tầm quan trọng,LABEL: positive | RATIONALE: nhấn mạnh tầm qua...,BARTPho
2,745,positive,negative,lo lắng và lo lắng,LABEL: negative | RATIONALE: lo lắng và lo lắng,BARTPho
3,1679,negative,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
4,519,negative,negative,tiêu xương,LABEL: negative | RATIONALE: tiêu xương,BARTPho
5,208,negative,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
6,553,neutral,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
7,1672,neutral,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
8,79,neutral,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,BARTPho
9,1558,neutral,negative,tình trạng bệnh lý tiêu cực,LABEL: negative | RATIONALE: tình trạng bệnh l...,BARTPho


In [ ]:
free_vram_strong(bartpho_tokenizer, bartpho_model)

**VIT5**

In [ ]:
import os, torch
from transformers import AutoModelForSeq2SeqLM, T5Tokenizer

# ===== load model from .pt =====
def load_vit5_from_pt(pt_path: str, base_model: str = "VietAI/vit5-base", device=None):
    assert os.path.exists(pt_path), f"Not found: {pt_path}"

    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    device = torch.device(device)

    # ViT5/SentencePiece: nên dùng slow tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=False)

    # Load base model (fp16 nếu CUDA để nhanh hơn)
    dtype = torch.float16 if device.type == "cuda" else torch.float32
    model = AutoModelForSeq2SeqLM.from_pretrained(base_model, torch_dtype=dtype)

    ckpt = torch.load(pt_path, map_location="cpu")
    if isinstance(ckpt, dict):
        state_dict = ckpt.get("model_state_dict") or ckpt.get("state_dict") or ckpt
    else:
        raise ValueError("File .pt không phải state_dict dict. (Bạn có thể đã save cả model object)")

    # dọn prefix (phòng khi save có 'model.' hoặc 'module.')
    clean = {}
    for k, v in state_dict.items():
        if k.startswith("model."):  k = k[len("model."):]
        if k.startswith("module."): k = k[len("module."):]
        clean[k] = v

    missing, unexpected = model.load_state_dict(clean, strict=False)
    model.to(device).eval()

    print(" Loaded ViT5 from .pt")
    print("  device:", device)
    print("  missing keys:", len(missing), "| unexpected keys:", len(unexpected))
    return tokenizer, model

In [ ]:
USE_PT = os.path.exists(vit5_pt_path)

if USE_PT:
    print(f"Loading ViT5 from .pt file: {vit5_pt_path}")
    _tok_src = vit5_path if os.path.isdir(vit5_path) else vit5_base
    vit5_tokenizer, vit5_model = load_seq2seq_from_pt(
        pt_path=vit5_pt_path,
        base_model_name=vit5_base,
        model_name="ViT5",
        tokenizer_name=_tok_src,
    )
else:
    print(f"Loading ViT5 from folder (safetensors): {vit5_path}")
    vit5_tokenizer, vit5_model = load_seq2seq_model(vit5_path, "ViT5")

df_vit5 = predict_seq2seq("ViT5", vit5_tokenizer, vit5_model, human_text, IDX)

print("\n--- ViT5 predictions (human_text) ---")
display(df_vit5)

Loading ViT5 from .pt file: /content/drive/MyDrive/Drive Seminar/Load_model/VIT5/vit5_finetuned.pt

Loading ViT5 from .pt: /content/drive/MyDrive/Drive Seminar/Load_model/VIT5/vit5_finetuned.pt


Loading weights:   0%|          | 0/260 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie shared.weight to encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The tied weights mapping and config for this model specifies to tie shared.weight to decoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


  Tokenizer loaded via T5Tokenizer ✓
  Loaded ✓  |  309.1M params  |  device=cuda:0

--- ViT5 predictions (human_text) ---


,id,true_label,pred_label,pred_rationale,raw_output,model
0,631,None,None,,,ViT5
1,1999,None,None,,,ViT5
2,626,None,None,,,ViT5
3,957,None,None,,,ViT5
4,538,None,None,,,ViT5
5,614,None,None,,,ViT5
6,42,None,None,,,ViT5
7,46,None,None,,,ViT5
8,2097,None,None,,,ViT5
9,2006,None,None,,,ViT5


In [ ]:

vit5_tokenizer, vit5_model = load_vit5_from_pt(vit5_pt_path, vit5_base_id)

df_vit5_human = predict_seq2seq("ViT5-ft", vit5_tokenizer, vit5_model, human_text, IDX, true_labels_human)
print("\n--- ViT5-ft predictions (human_text) ---")
display(df_vit5_human)





📦 Loading ViT5 finetuned (.pt):
/content/drive/MyDrive/Drive Seminar/Load_model/VIT5/vit5_finetuned.pt


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

✅ Loaded ViT5 from .pt
  device: cuda:0
  missing keys: 0 | unexpected keys: 0

--- ViT5-ft predictions (human_text) ---


,id,true_label,pred_label,pred_rationale,raw_output,model
0,631,positive,positive,có lợi cho sức khỏe,labelpositive; rationale có lợi cho sức khỏe,ViT5-ft
1,1999,positive,negative,sự thật bất lợi,labelnegative; rationale sự thật bất lợi,ViT5-ft
2,626,positive,neutral,tìm kiếm lời khuyên,labelneutral; rationale tìm kiếm lời khuyên,ViT5-ft
3,957,negative,neutral,tìm kiếm lời khuyên y tế,labelneutral; rationale tìm kiếm lời khuyên y tế,ViT5-ft
4,538,negative,negative,mối quan tâm về sức khỏe,labelnegative; rationale mối quan tâm về sức khỏe,ViT5-ft
5,614,negative,neutral,mô tả khách quan,labelneutral; rationale mô tả khách quan,ViT5-ft
6,42,neutral,neutral,mô tả khách quan,labelneutral; rationale mô tả khách quan,ViT5-ft
7,46,neutral,neutral,mô tả khách quan,labelneutral; rationale mô tả khách quan,ViT5-ft
8,2097,neutral,positive,chất lượng cuộc sống bình thường như bao người...,labelpositive; rationale chất lượng cuộc sống ...,ViT5-ft
9,2006,neutral,negative,sự thật bất lợi,labelnegative; rationale sự thật bất lợi,ViT5-ft


In [ ]:
df_vit5_phow = predict_seq2seq("ViT5-ft", vit5_tokenizer, vit5_model, phowhisper_transcripts, sample_ids, true_labels)
print("\n--- ViT5-ft predictions (PhoWhisper transcripts) ---")
display(df_vit5_phow)


--- ViT5-ft predictions (PhoWhisper transcripts) ---


,id,true_label,pred_label,pred_rationale,raw_output,model
0,794,negative,negative,dễ bị tổn thương cảm xúc,labelnegative; rationale dễ bị tổn thương cảm xúc,ViT5-ft
1,1088,positive,neutral,mô tả khách quan,labelneutral; rationale mô tả khách quan,ViT5-ft
2,745,negative,negative,dấu hiệu cảm xúc tiêu cực,labelnegative; rationale dấu hiệu cảm xúc tiêu...,ViT5-ft
3,1679,neutral,neutral,mô tả khách quan,labelneutral; rationale mô tả khách quan,ViT5-ft
4,519,negative,negative,vấn đề tiêu cực về tiêu cực,labelnegative; rationale vấn đề tiêu cực về ti...,ViT5-ft
5,208,neutral,neutral,mô tả khách quan,labelneutral; rationale mô tả khách quan,ViT5-ft
6,553,positive,positive,mong muốn tiến bộ,labelpositive; rationale mong muốn tiến bộ,ViT5-ft
7,1672,neutral,neutral,mô tả khách quan,labelneutral; rationale mô tả khách quan,ViT5-ft
8,79,positive,positive,địa chỉ uy tín cho phép trải nghiệm,labelpositive; rationaleđịa chỉ uy tín cho phé...,ViT5-ft
9,1558,neutral,negative,khó kiểm soát,labelnegative; rationale khó kiểm soát,ViT5-ft


In [ ]:

df_vit5_sherpa = predict_seq2seq("ViT5-ft", vit5_tokenizer, vit5_model, sherpa_transcripts, sample_ids, true_labels)
print("\n--- ViT5-ft predictions (Sherpa transcripts) ---")
display(df_vit5_sherpa)


--- ViT5-ft predictions (Sherpa transcripts) ---


,id,true_label,pred_label,pred_rationale,raw_output,model
0,794,negative,negative,dễ bị tổn thương bởi cảm xúc.,labelnegative; rationale dễ bị tổn thương bởi ...,ViT5-ft
1,1088,positive,positive,nhấn mạnh tầm quan trọng,labelpositive; rationale nhấn mạnh tầm quan trọng,ViT5-ft
2,745,negative,negative,dấu hiệu cảm xúc tiêu cực,labelnegative; rationale dấu hiệu cảm xúc tiêu...,ViT5-ft
3,1679,neutral,neutral,mô tả khách quan,labelneutral; rationale mô tả khách quan,ViT5-ft
4,519,negative,negative,vấn đề sức khỏe,labelnegative; rationale vấn đề sức khỏe,ViT5-ft
5,208,neutral,neutral,mô tả khách quan,labelneutral; rationale mô tả khách quan,ViT5-ft
6,553,positive,positive,mong muốn tiến bộ,labelpositive; rationale mong muốn tiến bộ,ViT5-ft
7,1672,neutral,neutral,mô tả khách quan,labelneutral; rationale mô tả khách quan,ViT5-ft
8,79,positive,positive,địa chỉ uy tín,labelpositive; rationale địa chỉ uy tín,ViT5-ft
9,1558,neutral,negative,kiểm soát khó khăn,labelnegative; rationale kiểm soát khó khăn,ViT5-ft


In [ ]:
free_vram_strong(vit5_tokenizer, vit5_model)

# **Decoder**

In [ ]:
ADAPTER_DIRS = {
    "qwen":   "Qwen3",
    "vistral":"vistralvistral7B",
    "llama":  "Llama3",
}

In [ ]:
SYS_PROMPT = "Bạn là một chuyên gia phân tích cảm xúc tiếng Việt. Hãy phân loại cảm xúc và đưa ra lý giải chi tiết."

def build_user_prompt(text: str):
    return (
        "Phân tích cảm xúc của câu sau:\n"
        f"'{text}'\n"
        "Trả về định dạng: LABEL: <negative|neutral|positive> | "
        "RATIONALE: <giải thích lý do ngắn gọn bằng tiếng Việt>"
    )

def build_chat_prompt(tokenizer, model_key: str, text: str):
    msgs = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": build_user_prompt(text)},
    ]
    # critical: let tokenizer decide correct template
    return tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)

### Utils

In [ ]:
def strip_think_blocks(s: str) -> str:
    """
    Remove Qwen reasoning blocks like:
    <think> ... </think>
    Also handles multiline, multiple blocks.
    """
    if s is None:
        return s
    s = str(s)
    s = re.sub(r"<think>[\s\S]*?</think>", "", s, flags=re.IGNORECASE)
    # dọn dòng trống dư
    s = re.sub(r"\n{3,}", "\n\n", s).strip()
    return s

In [ ]:
# Terminators per model (for Llama)
def get_terminators(tokenizer, model_key: str):
    eos = tokenizer.eos_token_id

    if model_key == "llama":
        eot = tokenizer.convert_tokens_to_ids("<|eot_id|>")
        if eot is None or eot < 0:
            return eos
        return [eos, eot]

    if model_key == "qwen":
        # Qwen eos is typically <|im_end|>. Also stop on pad if different.
        term = [eos]
        if tokenizer.pad_token_id is not None and tokenizer.pad_token_id != eos:
            term.append(tokenizer.pad_token_id)
        return term

    return eos

# Tokenizer source auto-detect (for Qwen)
def has_tokenizer_files(path: str) -> bool:
    files = [
        "tokenizer.json", "tokenizer.model", "tokenizer_config.json",
        "special_tokens_map.json", "vocab.json", "merges.txt"
    ]
    return any(os.path.exists(os.path.join(path, f)) for f in files)

def inspect_tokenizer(tokenizer, model_key: str):
    print("\n=== TOKENIZER INSPECT:", model_key, "===")
    print("bos:", tokenizer.bos_token, tokenizer.bos_token_id)
    print("eos:", tokenizer.eos_token, tokenizer.eos_token_id)
    print("pad:", tokenizer.pad_token, tokenizer.pad_token_id)
    print("has chat_template:", hasattr(tokenizer, "chat_template") and tokenizer.chat_template is not None)
    if hasattr(tokenizer, "chat_template") and tokenizer.chat_template:
        # print first ~400 chars only
        print("chat_template head:", tokenizer.chat_template[:400].replace("\n","\\n"))

### Load decoder + 4bit

In [ ]:
def load_decoder_by_adapter(adapter_dir: str, model_key: str):
    cfg_path = os.path.join(adapter_dir, "adapter_config.json")
    if not os.path.exists(cfg_path):
        raise ValueError(f"Missing adapter_config.json in: {adapter_dir}")

    with open(cfg_path, "r", encoding="utf-8") as f:
        cfg = json.load(f)

    base_id = cfg.get("base_model_name_or_path")
    if not base_id:
        raise ValueError(f"adapter_config.json missing base_model_name_or_path: {cfg_path}")

    print(f"\nAdapter: {adapter_dir}")
    print(f"Base:    {base_id}")
    print(f"LoRA:    r={cfg.get('r')} alpha={cfg.get('lora_alpha')}")

    compute_dtype = torch.bfloat16 if (torch.cuda.is_available() and torch.cuda.is_bf16_supported()) else torch.float16
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
    )

    tok_source = adapter_dir if has_tokenizer_files(adapter_dir) else base_id
    print(f"Tokenizer source : {tok_source}")

    tokenizer = AutoTokenizer.from_pretrained(
        tok_source,
        use_fast=True,
        trust_remote_code=True,
        token=True,
    )

    # Ensure pad/eos are set properly
    if tokenizer.pad_token_id is None:
        # if no pad token, fall back to eos (rare for these)
        tokenizer.pad_token = tokenizer.eos_token

    tokenizer.padding_side = "left"  # match eval style in your train scripts

    inspect_tokenizer(tokenizer, model_key)

    base = AutoModelForCausalLM.from_pretrained(
        base_id,
        device_map="auto",
        trust_remote_code=True,
        token=True,
        quantization_config=bnb_config,
    )

    model = PeftModel.from_pretrained(base, adapter_dir)
    model.eval()
    return tokenizer, model, base, base_id

In [ ]:
# Generation (Qwen anti-loop + correct pad_token_id)
@torch.no_grad()
def generate_new_text(tokenizer, model, prompt: str, eos_token_id, model_key: str, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    gen_kwargs = dict(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=eos_token_id,
        return_dict_in_generate=True,
    )

    #  Qwen often loops punctuation -> add light anti-loop only for Qwen
    if model_key == "qwen":
        gen_kwargs.update(dict(
            repetition_penalty=1.05,
            no_repeat_ngram_size=3,
        ))

    out = model.generate(**gen_kwargs)

    seq = out.sequences[0]
    prompt_len = inputs["input_ids"].shape[-1]
    gen_tokens = seq[prompt_len:]
    gen_text = tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()
    return gen_text

### Inference

In [ ]:
def predict_model(
    model_key: str,
    df: list[str],
    idx_list,
    max_new_tokens=128,
    save_dir="predictions",
    debug_first_n=1,
):
    assert model_key in ADAPTER_DIRS, f"Unknown model_key={model_key}. Use one of: {list(ADAPTER_DIRS.keys())}"

    tok = mdl = base = None
    try:
        tok, mdl, base, base_id = load_decoder_by_adapter(ADAPTER_DIRS[model_key], model_key)
        terminators = get_terminators(tok, model_key)

        rows = []
        for pos, idx in enumerate(idx_list):
            r = df[pos]
            prompt = build_chat_prompt(tok, model_key, r)

            gen = generate_new_text(
                tok, mdl, prompt,
                eos_token_id=terminators,
                model_key=model_key,
                max_new_tokens=max_new_tokens
            )
            gen_clean = strip_think_blocks(gen)

            if debug_first_n and pos < debug_first_n:
                print("\n--- DEBUG PROMPT (tail 400 chars) ---")
                print(prompt[-400:])
                print("--- END PROMPT ---")
                print("\n--- DEBUG RAW OUTPUT ---")
                print(gen)
                print("--- END RAW OUTPUT ---\n")

            pred_label, pred_rationale = parse_label_rationale(gen_clean)

            rows.append({
                "id": int(idx),
                "true_label": true_labels_human[pos],
                "pred_label": normalize_label(pred_label),
                "pred_rationale": pred_rationale,
                "raw_output": gen_clean,
                "model": model_key,
            })

        out_df = pd.DataFrame(rows)
        os.makedirs(save_dir, exist_ok=True)
        out_path = f"{save_dir}/pred_{model_key}.csv"
        out_df.to_csv(out_path, index=False, encoding="utf-8-sig")
        print("Saved:", out_path)
        display(out_df)

        return out_df

    finally:
        free_vram_strong(tok, mdl, base)
        free_vram_strong()

In [ ]:
print("\n--- Qwen3-8B predictions (human_text) ---")
df_pred_qwen = predict_model("qwen", human_text, IDX, max_new_tokens=96, debug_first_n=1)


--- Qwen3-8B predictions (human_text) ---

Adapter: /content/drive/MyDrive/Drive Seminar/Load_model/Qwen3
Base:    Qwen/Qwen3-8B
LoRA:    r=8 alpha=16
Tokenizer source : /content/drive/MyDrive/Drive Seminar/Load_model/Qwen3

=== TOKENIZER INSPECT: qwen ===
bos: None None
eos: <|im_end|> 151645
pad: <|endoftext|> 151643
has chat_template: True
chat_template head: {%- if tools %}\n    {{- '<|im_start|>system\n' }}\n    {%- if messages[0].role == 'system' %}\n        {{- messages[0].content + '\n\n' }}\n    {%- endif %}\n    {{- "# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}\n    {%- for tool in tools %}\n        {{- "\n" }}\n        {{- tool | t


Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- DEBUG PROMPT (tail 400 chars) ---
hân tích cảm xúc tiếng Việt. Hãy phân loại cảm xúc và đưa ra lý giải chi tiết.<|im_end|>
<|im_start|>user
Phân tích cảm xúc của câu sau:
'ý thức và thói quen có lợi cho việc nâng cao sức đề kháng của người dân từng bước được nâng cao từ việc trang bị những'
Trả về định dạng: LABEL: <negative|neutral|positive> | RATIONALE: <giải thích lý do ngắn gọn bằng tiếng Việt><|im_end|>
<|im_start|>assistant

--- END PROMPT ---

--- DEBUG RAW OUTPUT ---
<think>

</think>

LABEL: positive | Rationale:  nâng caosức khỏe cộng đồng.
--- END RAW OUTPUT ---

Saved: /content/drive/MyDrive/Drive Seminar/predictions/pred_qwen.csv


,id,true_label,pred_label,pred_rationale,raw_output,model
0,631,positive,positive,nâng caosức khỏe cộng đồng.,LABEL: positive | Rationale: nâng caosức khỏe...,qwen
1,1999,positive,neutral,mô tả khách quan về các lựa chọn làm đẹp tóc.,LABEL: neutral | Rationale: mô tả khách quan ...,qwen
2,626,positive,neutral,tìm kiếm thông tin khách quan.,LABEL: neutral | Rationale: tìm kiếm thông ti...,qwen
3,957,negative,neutral,tìm kiếm lời khuyên y tế.,LABEL: neutral | Rationale: tìm kiếm lời khuy...,qwen
4,538,negative,negative,so sánh tiêu cực về sự khác biệt.,LABEL: negative | Rationale: so sánh tiêu cực...,qwen
5,614,negative,neutral,mô tả khách quan về việc xử lý nhiệt độ cơ thể...,LABEL: neutral | Rationale: mô tả khách quan ...,qwen
6,42,neutral,neutral,mô tả khách quan của việc chọn câu hỏi.,LABEL: neutral | Rationale: mô tả khách quan ...,qwen
7,46,neutral,neutral,mô tả khách quan của thủ tục y tế.,LABEL: neutral | Rationale: mô tả khách quan ...,qwen
8,2097,neutral,positive,sự thật có lợi.,LABEL: positive | Rationale: sự thật có lợi.,qwen
9,2006,neutral,negative,tác động tiêu cực tới tóc.,LABEL: negative | Rationale: tác động tiêu cự...,qwen


In [ ]:
print("\n--- Vistral-7B predictions (human_text) ---")
df_pred_vistral = predict_model("vistral", human_text, IDX, max_new_tokens=96, debug_first_n=1)


--- Vistral-7B predictions (human_text) ---

Adapter: /content/drive/MyDrive/Drive Seminar/Load_model/VISTRAL7B/final_vistral_sentiment_5e-5
Base:    Viet-Mistral/Vistral-7B-Chat
LoRA:    r=16 alpha=32
Tokenizer source : /content/drive/MyDrive/Drive Seminar/Load_model/VISTRAL7B/final_vistral_sentiment_5e-5

=== TOKENIZER INSPECT: vistral ===
bos: <s> 1
eos: </s> 2
pad: <unk> 0
has chat_template: True
chat_template head: {% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') 


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/133 [00:00<?, ?B/s]


--- DEBUG PROMPT (tail 400 chars) ---
<s>[INST] <<SYS>>
Bạn là một chuyên gia phân tích cảm xúc tiếng Việt. Hãy phân loại cảm xúc và đưa ra lý giải chi tiết.
<</SYS>>

Phân tích cảm xúc của câu sau:
'ý thức và thói quen có lợi cho việc nâng cao sức đề kháng của người dân từng bước được nâng cao từ việc trang bị những'
Trả về định dạng: LABEL: <negative|neutral|positive> | RATIONALE: <giải thích lý do ngắn gọn bằng tiếng Việt> [/INST]
--- END PROMPT ---

--- DEBUG RAW OUTPUT ---
LABEL: positive | RATIONALE:  nâng cao sức khỏe
--- END RAW OUTPUT ---

Saved: /content/drive/MyDrive/Drive Seminar/predictions/pred_vistral.csv


,id,true_label,pred_label,pred_rationale,raw_output,model
0,631,positive,positive,nâng cao sức khỏe,LABEL: positive | RATIONALE: nâng cao sức khỏe,vistral
1,1999,positive,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,vistral
2,626,positive,neutral,tìm kiếm lời khuyên,LABEL: neutral | RATIONALE: tìm kiếm lời khuyên,vistral
3,957,negative,neutral,tìm kiếm lời khuyên y tế,LABEL: neutral | RATIONALE: tìm kiếm lời khuy...,vistral
4,538,negative,negative,tình trạng bệnh lý tiêu cực,LABEL: negative | RATIONALE: tình trạng bệnh ...,vistral
5,614,negative,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,vistral
6,42,neutral,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,vistral
7,46,neutral,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,vistral
8,2097,neutral,positive,chất lượng cuộc sống bình thường là có lợi.,LABEL: positive | RATIONALE: chất lượng cuộc ...,vistral
9,2006,neutral,neutral,mô tả khách quan,LABEL: neutral | RATIONALE: mô tả khách quan,vistral


In [ ]:
print("\n--- Llama3-8B predictions (human_text) ---")
df_pred_llama = predict_model("llama", human_text, IDX, max_new_tokens=96, debug_first_n=1)


--- Llama3-8B predictions (human_text) ---

Adapter: /content/drive/MyDrive/Drive Seminar/Load_model/Llama3
Base:    meta-llama/Llama-3.1-8B-Instruct
LoRA:    r=16 alpha=32
Tokenizer source : /content/drive/MyDrive/Drive Seminar/Load_model/Llama3

=== TOKENIZER INSPECT: llama ===
bos: <|begin_of_text|> 128000
eos: <|eot_id|> 128009
pad: <|eot_id|> 128009
has chat_template: True
chat_template head: {{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = "26 Jul 2024" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the sy


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- DEBUG PROMPT (tail 400 chars) ---
úc và đưa ra lý giải chi tiết.<|eot_id|><|start_header_id|>user<|end_header_id|>

Phân tích cảm xúc của câu sau:
'ý thức và thói quen có lợi cho việc nâng cao sức đề kháng của người dân từng bước được nâng cao từ việc trang bị những'
Trả về định dạng: LABEL: <negative|neutral|positive> | RATIONALE: <giải thích lý do ngắn gọn bằng tiếng Việt><|eot_id|><|start_header_id|>assistant<|end_header_id|>


--- END PROMPT ---

--- DEBUG RAW OUTPUT ---
LABEL: positive | RATIONALE:  nâng cao lợi ích sức khỏe.
--- END RAW OUTPUT ---

Saved: /content/drive/MyDrive/Drive Seminar/predictions/pred_llama.csv


,id,true_label,pred_label,pred_rationale,raw_output,model
0,631,positive,positive,nâng cao lợi ích sức khỏe.,LABEL: positive | RATIONALE: nâng cao lợi ích...,llama
1,1999,positive,neutral,mô tả khách quan về sự phổ biến của các salon ...,LABEL: neutral | RATIONALE: mô tả khách quan ...,llama
2,626,positive,neutral,tìm kiếm lời khuyên y tế khách quan.,LABEL: neutral | RATIONALE: tìm kiếm lời khuy...,llama
3,957,negative,neutral,tìm kiếm lời khuyên y tế khách quan.,LABEL: neutral | RATIONALE: tìm kiếm lời khuy...,llama
4,538,negative,negative,vấn đề sức khỏe tiêu cực được đề cập.,LABEL: negative | RATIONALE: vấn đề sức khỏe ...,llama
5,614,negative,neutral,mô tả khách quan về cách xử lý sốt thấp.,LABEL: neutral | RATIONALE: mô tả khách quan ...,llama
6,42,neutral,neutral,mô tả khách quan của thủ tục.,LABEL: neutral | RATIONALE: mô tả khách quan ...,llama
7,46,neutral,neutral,mô tả khách quan về cách nhận biết và phòng ngừa.,LABEL: neutral | RATIONALE: mô tả khách quan ...,llama
8,2097,neutral,positive,chất lượng cuộc sống bình thường là có lợi.,LABEL: positive | RATIONALE: chất lượng cuộc ...,llama
9,2006,neutral,neutral,mô tả khách quan các yếu tố gây hại cho tóc.,LABEL: neutral | RATIONALE: mô tả khách quan ...,llama
